In [1]:
import skfuzzy as fuzz
import numpy as np
import matplotlib.pyplot as plt
import json
import pandas as pd

In [2]:
my_data = np.genfromtxt('dataset/mat_csv.csv', delimiter=',')
my_data.shape

(48, 264)

In [5]:
i = 0
text_file = open("dataset/filenames_1.txt", "r")
lines = [line.rstrip('\n') for line in text_file.readlines()]
text_file.close()
for i in range(len(lines)):
    lines[i] = lines[i].replace("#", "/")

In [7]:
pattern_data = json.load(open('dataset/pattern.json', encoding="utf8"))  
pattern_dict = {}
i = 0
for i in range(264):
    flag = 0
    for j in range(len(pattern_data)):
        for k in range(len(pattern_data[j]['changedFiles'])):
            for key in pattern_data[j]['changedFiles'].keys():
                file_name = key.lstrip()
                file_name = file_name.rstrip()
                if file_name in lines[i]:
                    flag = 1
                    pattern_dict[i] = pattern_data[j]['repairPatterns']
                    #pattern_files.append(file_name)
    if flag == 0:
        print(lines[i])

In [8]:
classification_types = set()
for bug, patterns in pattern_dict.items():
    for pattern in patterns:
        if pattern not in classification_types:
            classification_types.add(pattern)

In [136]:
np.sum(u[:, 3])

0.9999999999999997

In [137]:
def find_cluster_dict(u, thres):
    clust_dict = {}
    for i in range(264):
        for j in range(cnum):
            if u[j, i] > thres:
                if j in clust_dict:
                        clust_dict[j].append(i)
                else:
                    clust_dict[j] = [i]
    return clust_dict

In [9]:
def find_cat_dict(clust_dict):
    category_dict = {}
    for cluster_num, bugs in clust_dict.items():
        for bug in bugs:
            if cluster_num in category_dict:
                patterns_before = category_dict[cluster_num]['pattern_count']
                for pattern in pattern_dict[bug]:
                    if pattern in classification_types:
                        category_dict[cluster_num]['pattern_count'] += 1
                        if pattern in category_dict[cluster_num]:
                            category_dict[cluster_num][pattern] += 1
                        else:
                            category_dict[cluster_num][pattern] = 1
                if category_dict[cluster_num]['pattern_count'] != patterns_before:
                    category_dict[cluster_num]['bug_count'] += 1
            else:
                category_dict[cluster_num] = {}
                category_dict[cluster_num]['pattern_count'] = 0
                patterns_before = category_dict[cluster_num]['pattern_count']
                for pattern in pattern_dict[bug]:
                    if pattern in classification_types:
                        category_dict[cluster_num]['pattern_count'] += 1
                        category_dict[cluster_num][pattern] = 1
                if category_dict[cluster_num]['pattern_count'] != patterns_before:
                    category_dict[cluster_num]['bug_count'] = 1
    return category_dict

In [119]:
def find_cluster_patterns(category_dict):
    cluster_patterns = {}
    for cluster_num in category_dict:
        if category_dict[cluster_num]['bug_count'] > 1:
            for pattern in category_dict[cluster_num]:
                if pattern != 'pattern_count' and pattern != 'bug_count':
                    tmp = category_dict[cluster_num][pattern] / category_dict[cluster_num]['bug_count']
                    #print(tmp)
                    if tmp >= 0.5:
                        if pattern in cluster_patterns:
                            if tmp > cluster_patterns[pattern][1]:
                                cluster_patterns[pattern] = (cluster_num,tmp)
                        else:
                            cluster_patterns[pattern] = (cluster_num,tmp)
    return cluster_patterns

In [176]:
for i in range(264):
    for j in range(14):
        if u[j][i] >= 0.3:
            print(u[j][i], i)
            break

0.9999999813739923 0
0.9997638229770787 1
0.9876299770006925 2
0.9986323408864646 3
0.9999999996093971 4
0.9836909537843892 5
0.9934127060601843 6
0.9999999837522826 7
0.9999807226145883 8
0.9999998652737255 9
0.99997880404391 10
0.9986989839209187 11
0.999999751388643 12
0.999999999995761 13
0.9993695676492897 14
0.5521118434996621 15
0.9948845157636532 16
0.9095887376365474 17
0.9999999370640407 18
0.9999988328798116 19
0.9999988099983642 20
0.4793178481511658 21
0.9999995668567165 22
0.9999999913427039 23
0.9863661866528749 24
0.9932620556888186 25
0.9999952489545688 26
0.999999999999762 27
0.9999999837522826 28
0.9999999927681691 29
0.9993780042949781 30
0.9999988099983642 31
0.9910737843483385 32
0.9999780217505196 33
0.9998909984003035 34
0.9999999837522826 35
0.7653241095956407 36
0.9999999994896136 37
0.9999668104553235 38
0.9999965304845394 39
0.9999998080461983 40
0.9999999837522826 41
0.9999988099983642 42
0.9998303569437006 43
0.9764731226332282 44
0.999962704451098 45
0.99

In [55]:
np.round(np.linspace(0.01, 0.5, 21), 2)

array([0.01, 0.03, 0.06, 0.08, 0.11, 0.13, 0.16, 0.18, 0.21, 0.23, 0.26,
       0.28, 0.3 , 0.33, 0.35, 0.38, 0.4 , 0.43, 0.45, 0.48, 0.5 ])

In [59]:
cnums = [i for i in range(10, 45)]
thres = np.round(np.linspace(0.01, 0.5, 21), 2)
for cnum in cnums:
    cntr, u, u_0, d, jm, p, fpc = fuzz.cmeans(my_data, cnum, 1.1, error = 0.0001, maxiter = 10000)
    for t in thres:
        #print(t)
        clust_dict = find_cluster_dict(u, t)
        category_dict = find_cat_dict(clust_dict)
        cluster_patterns = find_cluster_patterns(category_dict)
        cluster_set = set()
        for pattern in cluster_patterns:
            cluster_set.add(cluster_patterns[pattern][0])
        #print(len(cluster_set))
        if(len(cluster_set) / cnum >= 0.5):
            print(len(cluster_set) / cnum)
            print("************************************************")
            print("Parameters " + str(cnum) + " " + str(t) + " " + str(len(cluster_set)))
            for pattern in cluster_patterns:
                print('')
                print('\tCluster Number',cluster_patterns[pattern][0],'Pattern',pattern,'Confidence',cluster_patterns[pattern][1])
            print("************************************************")

0.5
************************************************
Parameters 10 0.16 5

	Cluster Number 4 Pattern singleLine Confidence 0.5363636363636364

	Cluster Number 3 Pattern condBlockOthersAdd Confidence 0.6

	Cluster Number 3 Pattern missComp Confidence 0.5

	Cluster Number 3 Pattern condBlockRetAdd Confidence 0.5

	Cluster Number 0 Pattern wrongComp Confidence 0.5

	Cluster Number 8 Pattern wrapsIfElse Confidence 0.5

	Cluster Number 5 Pattern expArithMod Confidence 0.5
************************************************
0.5
************************************************
Parameters 10 0.18 5

	Cluster Number 4 Pattern singleLine Confidence 0.5321100917431193

	Cluster Number 3 Pattern condBlockOthersAdd Confidence 0.6

	Cluster Number 3 Pattern missComp Confidence 0.5

	Cluster Number 3 Pattern condBlockRetAdd Confidence 0.5

	Cluster Number 0 Pattern wrongComp Confidence 0.5

	Cluster Number 8 Pattern wrapsIfElse Confidence 0.5

	Cluster Number 5 Pattern expArithMod Confidence 0.5
*******

In [138]:
cnums = [14]
thres = np.round(np.linspace(0.01, 0.5, 21), 2)
seed_vals = [i for i in range(100)]
for cnum in cnums:
    for s in seed_vals:
        cntr, u, u_0, d, jm, p, fpc = fuzz.cmeans(my_data, cnum, 1.1, error = 0.0001, maxiter = 10000, seed = s)
        for t in thres:
            #print(t)
            clust_dict = find_cluster_dict(u, t)
            category_dict = find_cat_dict(clust_dict)
            cluster_patterns = find_cluster_patterns(category_dict)
            cluster_set = set()
            for pattern in cluster_patterns:
                cluster_set.add(cluster_patterns[pattern][0])
            #print(len(cluster_set))
            if(len(cluster_set) / cnum >= 0.5):
                print(len(cluster_set) / cnum)
                print("************************************************")
                print("Parameters " + str(cnum) + " " + str(t) + " " + str(len(cluster_set)) + " seed " + str(s))
                clus_conf = 0
                for pattern in cluster_patterns:
                    if cluster_patterns[pattern][1] > 0.5:
                        print('')
                        print('\tCluster Number',cluster_patterns[pattern][0],'Pattern',pattern,'Confidence',cluster_patterns[pattern][1])
                        clus_conf += cluster_patterns[pattern][1]
                print("Sum " + str(clus_conf))
                print("************************************************")

0.5
************************************************
Parameters 14 0.16 7 seed 15

	Cluster Number 7 Pattern singleLine Confidence 0.5321100917431193

	Cluster Number 0 Pattern condBlockOthersAdd Confidence 0.6666666666666666

	Cluster Number 0 Pattern missComp Confidence 0.6666666666666666

	Cluster Number 5 Pattern condBlockRetAdd Confidence 0.6666666666666666
Sum 2.532110091743119
************************************************
0.5
************************************************
Parameters 14 0.18 7 seed 15

	Cluster Number 7 Pattern singleLine Confidence 0.5277777777777778

	Cluster Number 0 Pattern condBlockOthersAdd Confidence 0.6666666666666666

	Cluster Number 0 Pattern missComp Confidence 0.6666666666666666

	Cluster Number 2 Pattern wrongComp Confidence 0.5555555555555556

	Cluster Number 5 Pattern condBlockRetAdd Confidence 0.6666666666666666
Sum 3.0833333333333335
************************************************
0.5
************************************************
Paramet

In [139]:
cntr, u, u_0, d, jm, p, fpc = fuzz.cmeans(my_data, 14, 1.1, error = 0.0001, maxiter = 10000, seed = 53)
t = 0.3
clust_dict = find_cluster_dict(u, t)
category_dict = find_cat_dict(clust_dict)
cluster_patterns = find_cluster_patterns(category_dict)
cluster_set = set()
for pattern in cluster_patterns:
    cluster_set.add(cluster_patterns[pattern][0])
if(len(cluster_set) / cnum >= 0.5):
    print(len(cluster_set) / cnum)
    print("************************************************")
    print("Parameters " + str(cnum) + " " + str(t) + " " + str(len(cluster_set)) + " seed " + str(seed))
    clus_conf = 0
    for pattern in cluster_patterns:
        #if cluster_patterns[pattern][1] > 0.5:
        print('')
        print('\tCluster Number',cluster_patterns[pattern][0],'Pattern',pattern,'Confidence',cluster_patterns[pattern][1])
        clus_conf += cluster_patterns[pattern][1]
    print("Sum " + str(clus_conf))
    print("************************************************")

0.5
************************************************
Parameters 14 0.3 7 seed 49

	Cluster Number 5 Pattern singleLine Confidence 0.5130434782608696

	Cluster Number 10 Pattern condBlockOthersAdd Confidence 0.6666666666666666

	Cluster Number 10 Pattern missComp Confidence 0.6666666666666666

	Cluster Number 0 Pattern wrongComp Confidence 1.0

	Cluster Number 11 Pattern expLogicExpand Confidence 0.6

	Cluster Number 9 Pattern condBlockRetAdd Confidence 0.6666666666666666

	Cluster Number 0 Pattern expArithMod Confidence 1.0

	Cluster Number 8 Pattern expLogicMod Confidence 0.6666666666666666

	Cluster Number 8 Pattern wrongVarRef Confidence 0.6666666666666666

	Cluster Number 0 Pattern blockRemove Confidence 0.5

	Cluster Number 0 Pattern condBlockRem Confidence 0.5

	Cluster Number 0 Pattern wrongMethodRef Confidence 0.5

	Cluster Number 12 Pattern initFix Confidence 0.5

	Cluster Number 12 Pattern wrapsIfElse Confidence 0.5
Sum 8.946376811594202
**************************************

In [159]:
cnt_both = 0
cnt_cond = 0
cnt_miss = 0
for i in range(264):
    patterns = pattern_dict[i]
    if 'condBlockOthersAdd' in patterns and 'missComp' in patterns:
        print(i)
        cnt_both += 1
        cnt_cond += 1
        cnt_miss += 1
    elif 'condBlockOthersAdd' in patterns:
        cnt_cond += 1
    elif 'missComp' in patterns:
        cnt_miss += 1

0
10
16
23
24
76
93
109
111
115
120
142
150
151
153
155
157
163
172
203
221


In [143]:
print(cnt_both, cnt_cond, cnt_miss)

21 43 54


In [152]:
cnt_both_i_wraps = 0
cnt_wrongComp = 0
cnt_expArithMod = 0
for i in range(264):
    patterns = pattern_dict[i]
    if 'expArithMod' in patterns and 'wrongComp' in patterns:
        print(i)
        cnt_both_i_wraps += 1
        cnt_expArithMod += 1
        cnt_wrongComp += 1
    elif 'wrongComp' in patterns:
        cnt_wrongComp += 1
    elif 'expArithMod' in patterns:
        cnt_expArithMod += 1

104
132
138
145
154
166
167
176
188
211
213
215
227
235
236


In [151]:
print(cnt_both_i_wraps, cnt_expArithMod, cnt_wrongComp)

15 27 38


In [174]:
lines[203]

'7bbddc_350b2b_src/main/java/org/apache/commons/math3/linear/OpenMapRealVector.java'

In [175]:
pattern_dict[203]

['condBlockOthersAdd', 'missComp', 'wrongComp']